In [1]:
import matplotlib.pyplot as plt

In [2]:
import numpy as np
from pkg.utils import *
import socket
import time as timer
import json

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)


In [3]:
# 서버의 주소입니다. hostname 또는 ip address를 사용할 수 있습니다.
HOST = '192.168.0.172'  
# 서버에서 지정해 놓은 포트 번호입니다. 
PORT = 1189       


In [44]:
def send_recv(sdict):
    # 소켓 객체를 생성합니다. 
    # 주소 체계(address family)로 IPv4, 소켓 타입으로 TCP 사용합니다.  
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # 지정한 HOST와 PORT를 사용하여 서버에 접속합니다. 
    client_socket.connect((HOST, PORT))

    rdict = {}
    try:
        # 메시지를 전송합니다. 
#         sdict = {'qval':[1000.0]*7}
        sjson = json.dumps(sdict, cls=NumpyEncoder)
        sbuff = sjson.encode()
        client_socket.sendall(sbuff)

        # 메시지를 수신합니다. 
        data = client_socket.recv(1024)
#         print('Received', repr(data.decode()))
        rjson = "".join(map(chr, data))
        rdict = json.loads(rjson)
#         print(rdict)
    finally:
        # 소켓을 닫습니다.
        client_socket.close()
    return rdict

In [251]:
gtimer = GlobalTimer.instance()
gtimer.reset()

In [252]:
print(send_recv({'alpha_lpf':-1}))

{'alpha_lpf': 0.8}


In [253]:
rdict = send_recv({'alpha_lpf':0.8})

In [254]:
print(send_recv({'k_gain':-1}))

{'k_gain': 10.0}


In [255]:
rdict = send_recv({'k_gain':30.0})

In [256]:
print(send_recv({'d_gain':-1}))

{'d_gain': 0.1}


In [257]:
rdict = send_recv({'d_gain':5.0})

In [258]:
rdict = send_recv({'qcur':0})
print(rdict)

{'qcur': [0.14498284410488282, -0.4096345482205677, -0.17782473316974146, -2.0132755087403806, 0.06757280413309712, 2.210081308417844, 0.6818432107748059]}


In [259]:
traj_sample = []
DELTA = np.deg2rad(-30)
N_steps = 100
DT = 1e-2
t0 = timer.time()
rdict = send_recv({'qcur':0})
qval0 = rdict['qcur']
for iii in range(100):
    gtimer.tic("step")
    while timer.time()-t0<DT:
        timer.sleep(1e-3)
    t0 = timer.time()
    
    qval = np.add(qval0,DELTA*iii/N_steps).tolist()
    sdict = {'qval':qval}
    rdict = send_recv(sdict)
    traj_sample += rdict['qval']
    gtimer.toc("step")
print(gtimer)

step: 	1072.0 ms/100 = 10.72 ms 



In [260]:
traj_sample = []
DELTA = np.deg2rad(30)
N_steps = 100
DT = 1e-2
t0 = timer.time()
rdict = send_recv({'qcur':0})
qval0 = rdict['qcur']
for iii in range(100):
    gtimer.tic("step")
    while timer.time()-t0<DT:
        timer.sleep(1e-3)
    t0 = timer.time()
    
    qval = np.add(qval0,DELTA*iii/N_steps).tolist()
    sdict = {'qval':qval}
    rdict = send_recv(sdict)
    traj_sample += rdict['qval']
    gtimer.toc("step")
print(gtimer)

step: 	2139.0 ms/200 = 10.697 ms 



In [2]:
from nrmkindy.indy_script import *